### Bir geliştiricinin genel olarak, şu ana kadarki kariyerinden memnuniyet derecesinin (```CareerSat```) tahminlenmesi

In [2]:
import pandas as pd
import numpy as np
df_19=pd.read_csv("survey_results_public.csv")

In [3]:
df_19.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [4]:
S2df=df_19[['CareerSat','ConvertedComp','JobSat','Employment','EdLevel','MainBranch']]
S2df=S2df.dropna()

In [5]:
#Sutun işlemleri.Bu soruda ilk sorudan farklı olarak get_dummies ile yaptım.
JobSat=pd.get_dummies(S2df["JobSat"])
Employment=pd.get_dummies(S2df["Employment"])
MainBranch=pd.get_dummies(S2df["MainBranch"])
#Hobbyist=pd.get_dummies(S2df["Hobbyist"],drop_first=True)
#LastHireDate=pd.get_dummies(S2df["LastHireDate"])
EdLevel=pd.get_dummies(S2df["EdLevel"])
CareerSat=S2df["CareerSat"]
ConvertedComp=S2df["ConvertedComp"]
S2df=pd.concat([ConvertedComp,JobSat,CareerSat,Employment,EdLevel,MainBranch],axis=1)
X=S2df.drop(["CareerSat"],axis=1)
y=S2df["CareerSat"]

In [6]:
#Verileri ayırma işlemi.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#Kullanacağımız metotu belirledik.Bu soruda en aşağıda seçtiğim yöntemin en iyi yöntem olduğünu en azından özniteliklerim için 
#test etmek için en aşağıda vermiş olduğum test metotu ile karşılaştırma yaptım.
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(criterion='entropy')

In [8]:
#cv argümanını 5 vererek k-fold sayısını ayarlamış oluyoruz ve best_score çalıştırdığımız zaman K-fold 'a göre sonuç vermekte
#aşağıda 
from sklearn.model_selection import GridSearchCV
params = {'max_depth': [4,5,6], 'min_samples_leaf': [1,2,3], 'min_samples_split': [2,3,4]}
grid_search = GridSearchCV(estimator=dt_model, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(criterion='entropy'),
             param_grid={'max_depth': [4, 5, 6], 'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 3, 4]})

In [9]:
#GridSearchCV Skoru
grid_search.best_score_

0.5748328656931135

In [10]:
#y_pred değişkenine predict ettim.
y_pred = grid_search.predict(X_test)

In [11]:
#GridSearchCV skorundan hariç kendimde skorları predict üzerinden görüntüledim.
# Test verisindeki değerler ile 100 de 80 üzerinden yapılan tahminler karşılaştırılıp sonuç aşşağıda gösterdim.
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
acc = accuracy_score(y_test, y_pred)
pre = precision_score(y_test, y_pred, average='micro')
rec = recall_score(y_test, y_pred, average='micro')
f1 = f1_score(y_test, y_pred, average='micro')
acc, pre, rec, f1

(0.5817750615707379,
 0.5817750615707379,
 0.5817750615707379,
 0.5817750615707379)

In [12]:
S2df.head()

,ConvertedComp,Neither satisfied nor dissatisfied,Slightly dissatisfied,Slightly satisfied,Very dissatisfied,Very satisfied,CareerSat,Employed full-time,Employed part-time,"Independent contractor, freelancer, or self-employed",...,"Bachelor’s degree (BA, BS, B.Eng., etc.)",I never completed any formal education,"Master’s degree (MA, MS, M.Eng., MBA, etc.)","Other doctoral degree (Ph.D, Ed.D., etc.)",Primary/elementary school,"Professional degree (JD, MD, etc.)","Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)",Some college/university study without earning a degree,I am a developer by profession,"I am not primarily a developer, but I write code sometimes as part of my work"
2,8820.0,0,0,1,0,0,Slightly satisfied,1,0,0,...,1,0,0,0,0,0,0,0,0,1
3,61000.0,0,0,1,0,0,Very satisfied,1,0,0,...,1,0,0,0,0,0,0,0,1,0
5,366420.0,0,0,1,0,0,Very satisfied,1,0,0,...,1,0,0,0,0,0,0,0,0,1
8,95179.0,0,0,1,0,0,Slightly satisfied,1,0,0,...,0,0,0,0,0,0,0,1,1,0
9,13293.0,0,1,0,0,0,Slightly dissatisfied,1,0,0,...,0,0,1,0,0,0,0,0,1,0


In [13]:
#İlk satırdaki verinin bilgilerini girdiğimizde doğru sonuç aldığımzı aşağıda görmekteyiz.
yeni_veri=np.array([[8820,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1]])
grid_search.predict(yeni_veri)

array(['Slightly satisfied'], dtype=object)